In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
import time
random.seed(0)

In [2]:
# 导入Fmnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [6]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8293
SVD: 0.8224
NetD, Acc at first 0.8394


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/it]


----------------------------
Train_z, Test_z
PCA: 0.8089999999999999
SVD: 0.8044
NetG, Acc at first 0.8476


In [7]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 5

for epoch in range(num_steps):
    if 1==1:
        
        #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        #radius_list.append(radius)
        #print(radius)

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()

            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        

1.653596263293532
tensor(-16.6684, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-6.4344, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.5915, device='cuda:0', grad_fn=<NegBackward0>), tensor(-37.1868, device='cuda:0', grad_fn=<AddBackward0>)]
55.85819182341144
tensor(-12.7055, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.9128, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.1868, device='cuda:0', grad_fn=<NegBackward0>), tensor(-35.8131, device='cuda:0', grad_fn=<AddBackward0>)]
17.609105900247496
tensor(-12.3337, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.9128, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.8202, device='cuda:0', grad_fn=<NegBackward0>), tensor(-35.6238, device='cuda:0', grad_fn=<AddBackward0>)]
1.9693679158555655
tensor(-17.0886, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-7.3440, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.1937, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.8425, device='cuda:0', grad_fn=<AddBac

0.49040556476338193
tensor(-47.8240, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.1610, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.5695, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.1131, device='cuda:0', grad_fn=<AddBackward0>)]
0.4633517437304587
tensor(-47.6583, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.1610, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.4716, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.5811, device='cuda:0', grad_fn=<AddBackward0>)]
0.5219619763232969
tensor(-48.4426, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.5065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.7911, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.5410, device='cuda:0', grad_fn=<AddBackward0>)]
0.5179665897450073
tensor(-48.3030, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.5065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.7151, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.0161, device='cuda:0', grad_fn=

0.46860450558545147
tensor(-59.1896, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.9465, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.2868, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.0731, device='cuda:0', grad_fn=<AddBackward0>)]
0.4602013187006757
tensor(-59.6464, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.2663, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.4899, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.6207, device='cuda:0', grad_fn=<AddBackward0>)]
0.45841497035516765
tensor(-59.3890, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.2663, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.2011, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.8316, device='cuda:0', grad_fn=<AddBackward0>)]
0.43263864084975406
tensor(-59.8825, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.5033, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.6304, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.7478, device='cuda:0', grad_f

0.3998086228273155
tensor(-65.6627, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.4147, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.5915, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.2413, device='cuda:0', grad_fn=<AddBackward0>)]
0.3622723189660011
tensor(-65.6627, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.4147, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.8022, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.2455, device='cuda:0', grad_fn=<AddBackward0>)]
0.36701953119032593
tensor(-65.2021, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.2520, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.4335, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.5567, device='cuda:0', grad_fn=<AddBackward0>)]
0.3918135713469645
tensor(-65.2846, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.2520, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.5132, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.5695, device='cuda:0', grad_fn=

0.3686345202250334
tensor(-70.1292, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.8160, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.8943, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.1325, device='cuda:0', grad_fn=<AddBackward0>)]
0.39951485308878026
tensor(-69.9243, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.6256, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.7186, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.8555, device='cuda:0', grad_fn=<AddBackward0>)]
0.3949437727068548
tensor(-70.0221, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.6256, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.7794, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.0385, device='cuda:0', grad_fn=<AddBackward0>)]
0.3402342681207847
tensor(-70.4536, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.9583, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.1804, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.7246, device='cuda:0', grad_fn=

...

0.23095279780858508
tensor(-95.5651, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.7164, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.3259, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6866, device='cuda:0', grad_fn=<AddBackward0>)]
0.25444774237514156
tensor(-95.5188, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6928, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.2502, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7785, device='cuda:0', grad_fn=<AddBackward0>)]
0.22959397464404135
tensor(-95.5048, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6928, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.2692, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7208, device='cuda:0', grad_fn=<AddBackward0>)]
0.22484581910979626
tensor(-95.5832, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6914, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.3547, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7110, device='cuda:0', grad_

0.2435227742425525
tensor(-95.8657, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.8999, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.4033, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7552, device='cuda:0', grad_fn=<AddBackward0>)]
0.2412546575612902
tensor(-95.8586, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.8999, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.4180, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7171, device='cuda:0', grad_fn=<AddBackward0>)]
0.23861977755191002
tensor(-95.4357, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6383, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.2747, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6866, device='cuda:0', grad_fn=<AddBackward0>)]
0.2266314871915199
tensor(-95.3219, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6383, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1729, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6666, device='cuda:0', grad_fn=

0.22801861990364705
tensor(-96.0166, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.9749, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.5621, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6154, device='cuda:0', grad_fn=<AddBackward0>)]
0.17691882110483165
tensor(-96.1463, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.0172, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.8226, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3588, device='cuda:0', grad_fn=<AddBackward0>)]
0.2656883844077579
tensor(-96.1497, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.0172, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.5118, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.8601, device='cuda:0', grad_fn=<AddBackward0>)]
0.22421578579810517
tensor(-95.9989, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.9611, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.5931, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.5601, device='cuda:0', grad_f

0.24938658312228312
tensor(-96.3251, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1099, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6242, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.8059, device='cuda:0', grad_fn=<AddBackward0>)]
0.2158035995051424
tensor(-96.3121, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1099, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7452, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.5793, device='cuda:0', grad_fn=<AddBackward0>)]
0.2506561843998101
tensor(-95.8710, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.9470, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.3274, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.8160, device='cuda:0', grad_fn=<AddBackward0>)]
0.20562178266049744
tensor(-96.0490, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.9470, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7237, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.4599, device='cuda:0', grad_fn

0.2372026463885436
tensor(-96.2188, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.0467, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6483, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6884, device='cuda:0', grad_fn=<AddBackward0>)]
0.2632527383487766
tensor(-96.3474, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1408, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.5934, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.8463, device='cuda:0', grad_fn=<AddBackward0>)]
0.218609791402756
tensor(-96.4260, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1408, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.8348, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.5691, device='cuda:0', grad_fn=<AddBackward0>)]
0.2118557789661669
tensor(-96.3145, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1333, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7235, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.5804, device='cuda:0', grad_fn=<A

In [8]:
radius_list

[1.653596263293532,
 55.85819182341144,
 17.609105900247496,
 1.9693679158555655,
 39.10447611833945,
 1.068932255120621,
 18.247225042989434,
 1.3243323953254176,
 9.592802869477282,
 0.9150633875826496,
 1.049332746349577,
 0.7693179506641119,
 0.7624858157664572,
 0.7120519532433397,
 0.7983241763305862,
 0.5537503685785106,
 0.5772984090505414,
 0.6303144039440526,
 0.6084924234749628,
 0.6182642025055377,
 0.617428663364435,
 0.6021577071239849,
 0.7003659579698848,
 0.5852377563369315,
 0.5489228574007202,
 0.5947135346191872,
 0.5658534097505887,
 0.5579748247893955,
 0.4673755348828929,
 0.4980067250298854,
 0.4925018153444983,
 0.5570017404054532,
 0.5022293053127375,
 0.49040556476338193,
 0.4633517437304587,
 0.5219619763232969,
 0.5179665897450073,
 0.5105386405470416,
 0.47622760617853327,
 0.4734010263212929,
 0.5127751713583807,
 0.48510324034656305,
 0.48374328739399824,
 0.4761100425023211,
 0.4938586749388115,
 0.4756316001910571,
 0.49524440571929507,
 0.499396881945

In [9]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.57s/it]


----------------------------
Train_z, Test_z
PCA: 0.8438
SVD: 0.8409
NetD, Acc at first 0.8621


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.8356
SVD: 0.8321000000000001
NetG, Acc at first 0.8567


In [10]:
torch.save(netD, 'FMNIST_NetD_changing2-mode11.pth')
torch.save(netV, 'FMNIST_NetV_changing2-mode11.pth')
torch.save(netG, 'FMNIST_NetG_changing2-mode11.pth')

#netD = torch.load('FMNIST_NetD.pth')
#netV = torch.load('FMNIST_NetV.pth')
#netG = torch.load('FMNIST_NetG.pth')

#mode = 1
#gam3 = 1

In [11]:
r_list = [0,0.1,0.2,0.5,1,2,3,4,6,8,10]
sample_num = 20

In [12]:
# sample 20 times of NetG at final radius, NetD is fixed, length 0.1,0.2,0.5,1,2,4,6,8,10,15,20
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample = []
for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[1])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample.append(count/10000)
    
print(max(Acc_results_G_sample),min(Acc_results_G_sample),np.mean(Acc_results_G_sample),np.var(Acc_results_G_sample))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8354
SVD: 0.8339
NetG_fixD, Acc at sample 1 0.8574


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8358
SVD: 0.8333
NetG_fixD, Acc at sample 2 0.8577


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8366
SVD: 0.8331
NetG_fixD, Acc at sample 18 0.8585


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8362
SVD: 0.8336
NetG_fixD, Acc at sample 19 0.8583


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.8365
SVD: 0.8334
NetG_fixD, Acc at sample 20 0.8577
0.8587 0.8568 0.8577900000000002 2.508999999999969e-07


In [13]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample2 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[2])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample2.append(count/10000)
    
print(max(Acc_results_G_sample2),min(Acc_results_G_sample2),np.mean(Acc_results_G_sample2),np.var(Acc_results_G_sample2))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.8351999999999999
SVD: 0.833
NetG_fixD, Acc at sample 1 0.8581


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.8358
SVD: 0.8335
NetG_fixD, Acc at sample 2 0.8563


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8357
SVD: 0.8333
NetG_fixD, Acc at sample 18 0.8576


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8357
SVD: 0.8343
NetG_fixD, Acc at sample 19 0.8563


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.8345
SVD: 0.8325
NetG_fixD, Acc at sample 20 0.8579
0.8591 0.8559 0.857225 5.808749999999977e-07


In [14]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample3 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[3])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample3.append(count/10000)
    
print(max(Acc_results_G_sample3),min(Acc_results_G_sample3),np.mean(Acc_results_G_sample3),np.var(Acc_results_G_sample3))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8345
SVD: 0.8336
NetG_fixD, Acc at sample 1 0.8563


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8356
SVD: 0.8351999999999999
NetG_fixD, Acc at sample 2 0.8565


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8364
SVD: 0.8342
NetG_fixD, Acc at sample 18 0.858


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8351
SVD: 0.8334
NetG_fixD, Acc at sample 19 0.8557


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8337
SVD: 0.8316
NetG_fixD, Acc at sample 20 0.8534
0.8581 0.8534 0.8561 1.7380000000000003e-06


In [15]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample4 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[4])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample4.append(count/10000)
    
print(max(Acc_results_G_sample4),min(Acc_results_G_sample4),np.mean(Acc_results_G_sample4),np.var(Acc_results_G_sample4))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8323
SVD: 0.8289
NetG_fixD, Acc at sample 1 0.8516


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8375
SVD: 0.8336
NetG_fixD, Acc at sample 2 0.8544


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8349
SVD: 0.8313
NetG_fixD, Acc at sample 18 0.8535


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


----------------------------
Train_z, Test_z
PCA: 0.8334
SVD: 0.8311
NetG_fixD, Acc at sample 19 0.8534


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8363
SVD: 0.834
NetG_fixD, Acc at sample 20 0.8532
0.8544 0.8515 0.8529250000000002 6.908750000000073e-07


In [16]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample5 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[5])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample5.append(count/10000)
    
print(max(Acc_results_G_sample5),min(Acc_results_G_sample5),np.mean(Acc_results_G_sample5),np.var(Acc_results_G_sample5))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


----------------------------
Train_z, Test_z
PCA: 0.8295
SVD: 0.8243
NetG_fixD, Acc at sample 1 0.8385


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


----------------------------
Train_z, Test_z
PCA: 0.8280000000000001
SVD: 0.8237
NetG_fixD, Acc at sample 2 0.8429


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8287
SVD: 0.8266
NetG_fixD, Acc at sample 18 0.843


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/it]


----------------------------
Train_z, Test_z
PCA: 0.8255
SVD: 0.8267
NetG_fixD, Acc at sample 19 0.8413


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8311999999999999
SVD: 0.829
NetG_fixD, Acc at sample 20 0.8403
0.8446 0.8368 0.840495 5.007475000000007e-06


In [17]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample6 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[6])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample6.append(count/10000)
    
print(max(Acc_results_G_sample6),min(Acc_results_G_sample6),np.mean(Acc_results_G_sample6),np.var(Acc_results_G_sample6))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8117
SVD: 0.8056
NetG_fixD, Acc at sample 1 0.778


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8207
SVD: 0.8169
NetG_fixD, Acc at sample 2 0.794


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8240000000000001
SVD: 0.8204
NetG_fixD, Acc at sample 18 0.7869


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.825
SVD: 0.8156
NetG_fixD, Acc at sample 19 0.7883


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8197
SVD: 0.817
NetG_fixD, Acc at sample 20 0.7772
0.8158 0.7259 0.7817299999999999 0.0005025831000000002


In [18]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample7 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[7])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample7.append(count/10000)
    
print(max(Acc_results_G_sample7),min(Acc_results_G_sample7),np.mean(Acc_results_G_sample7),np.var(Acc_results_G_sample7))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.815
SVD: 0.8068
NetG_fixD, Acc at sample 1 0.4394


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8159
SVD: 0.8149
NetG_fixD, Acc at sample 2 0.5394


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8139000000000001
SVD: 0.8081
NetG_fixD, Acc at sample 18 0.4526


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8147
SVD: 0.808
NetG_fixD, Acc at sample 19 0.3652


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


----------------------------
Train_z, Test_z
PCA: 0.8168
SVD: 0.8088
NetG_fixD, Acc at sample 20 0.4
0.5739 0.3403 0.46339499999999995 0.003596151474999999


In [19]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample8 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[8])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample8.append(count/10000)
    
print(max(Acc_results_G_sample8),min(Acc_results_G_sample8),np.mean(Acc_results_G_sample8),np.var(Acc_results_G_sample8))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8084
SVD: 0.8106
NetG_fixD, Acc at sample 1 0.1442


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8077
SVD: 0.8032
NetG_fixD, Acc at sample 2 0.1638


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8071
SVD: 0.806
NetG_fixD, Acc at sample 18 0.1294


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.8075
SVD: 0.8035
NetG_fixD, Acc at sample 19 0.1422


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.48s/it]


----------------------------
Train_z, Test_z
PCA: 0.8045
SVD: 0.8023
NetG_fixD, Acc at sample 20 0.1508
0.172 0.0959 0.14664 0.00031166140000000005


In [20]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample9 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[9])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample9.append(count/10000)
    
print(max(Acc_results_G_sample9),min(Acc_results_G_sample9),np.mean(Acc_results_G_sample9),np.var(Acc_results_G_sample9))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8136
SVD: 0.8068
NetG_fixD, Acc at sample 1 0.1152


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8135
SVD: 0.8077
NetG_fixD, Acc at sample 2 0.0768


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8081
SVD: 0.8064
NetG_fixD, Acc at sample 18 0.1173


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8071
SVD: 0.8018000000000001
NetG_fixD, Acc at sample 19 0.0895


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


----------------------------
Train_z, Test_z
PCA: 0.81
SVD: 0.8055
NetG_fixD, Acc at sample 20 0.1168
0.1479 0.0768 0.10930000000000002 0.0003111330000000001


In [21]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample10 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[10])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample10.append(count/10000)
    
print(max(Acc_results_G_sample10),min(Acc_results_G_sample10),np.mean(Acc_results_G_sample10),np.var(Acc_results_G_sample10))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8097
SVD: 0.8078
NetG_fixD, Acc at sample 1 0.0863


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.8073
SVD: 0.806
NetG_fixD, Acc at sample 2 0.1003


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8159
SVD: 0.81
NetG_fixD, Acc at sample 18 0.1082


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8067
SVD: 0.8056
NetG_fixD, Acc at sample 19 0.0985


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


----------------------------
Train_z, Test_z
PCA: 0.8145
SVD: 0.8094
NetG_fixD, Acc at sample 20 0.1047
0.119 0.0805 0.10093 0.00010992909999999999
